In [4]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from urllib.error import HTTPError

In [10]:
#DEFAULT ELEMENT FOR RATING TO BE CALLED WHEN INCASE OF HTTP ERROR
default = """"<div class="rating">
      <span class="digit">0</span>
      <span class="digit">.</span>
      <span class="digit">0</span>
    </div>"""
#NUMBER OF PAGES
n = 8
movie_titles=[]
imax_schedule=[]
regular_schedule=[]
ratings=[]
rating_rough_extracts=[]
#LOOPING THROUGH PAGES
for i in range(1, n+1):
    #IDENTIFYING FIRST PAGE
    if (i == 1):
        # handle first page
        parse_hub=ureq("https://www.parsehub.com/sandbox/showtimes")
    #handles subsequent pages     
    parse_hub = ureq("https://www.parsehub.com/sandbox/showtimes" + "?page=%d" % i)
    ph_soup= soup(parse_hub,"lxml")
    movie_elements= ph_soup.find_all("a",{"class":"title"})
    #looping through lists movie_elements which is made up of lists
    for movie_element in movie_elements:
      #looping through inner list of movie_elements
      for sub_movie_element in movie_element:
        #appending movie extracted movie titles in list movie_titles
        movie_titles.append(sub_movie_element)
    #extracting movie schedules
    first_movie_showtime_imax = ph_soup.find_all("span",{"class":"borderbox showtime imax first"})
    other_movie_showtime_imax = ph_soup.find_all("span",{"class":"borderbox showtime imax other"})
    first_movie_showtimes_regular = ph_soup.find_all("span",{"class":"borderbox showtime regular first"})
    other_movie_showtimes_regular = ph_soup.find_all("span",{"class":"borderbox showtime regular other"})
    #adding imax showing time of first movie to a list
    imax_showtimes=[first_movie_showtime_imax[0].text]
    #looping through imax showing times of other movies and appending to imax_showingtimes
    for imax_time in other_movie_showtime_imax:
        imax_showtimes.append(imax_time.text)
    #looping through imax show times and appending to pre-defined imax_schedule list for final output
    for imax_showtime in imax_showtimes:
        imax_schedule.append(imax_showtime)
    #defining list for regular showing times
    reg_times=[]
    #appending regular showing times for first movie to reg_times
    for first_movie_showtime_regular in first_movie_showtimes_regular:
        reg_times.append(first_movie_showtime_regular.text)
    #appending regular showing times for other movies to reg_times
    for other_movie_showtime_regular in other_movie_showtimes_regular:
        reg_times.append(other_movie_showtime_regular.text)
    #defining dict 
    regular_times = {}
    #creating auxiliary lists to aid extraction of showing times from multi nested lists
    aux1=[]
    aux2=[]
    #defining iterable indexes  and appending to aux1
    for index1 in range(0,len(reg_times)-3,4):
        aux1.append(index1)
    #defining iterable indexes  and appending to aux2
    for index2 in range(4,len(reg_times)+1,4):
        aux2.append(index2)
    #defining number of iter which is dependent on len of auxilliary lists
    for number_iter in range(0,len(aux1)):
        #changing number_iter type to int
        number_iter_to_string=str(number_iter)
        #defining dict keys by merging number_iter
        time_values = reg_times[aux1[number_iter]:aux2[number_iter]]
        regular_schedule.append(time_values)
    individual_movie_links = ph_soup.find_all("a",{"class":"title"})
    for individual_movie_link in individual_movie_links:
        #declaring exception
        try:
            #formatting indvividual movie link
            indie_link = "https://www.parsehub.com/" + individual_movie_link.get("href").replace(" ","%20")
            movie_html =ureq(indie_link)
      #if HTTPError is received replace movie with default(a predefined string)
        except HTTPError as e:
            movie_html= default
        movie_soup = soup(movie_html,"lxml")
        movies = movie_soup.find_all("span",{"class":"digit"})
        #appending extracts to predefined list
        rating_rough_extracts.append(movies)
extracts=[]
#iterating through nested list to extract values
for rating_rough_extract in rating_rough_extracts:
    for extract in rating_rough_extract:
        extracts.append(extract)
#formating values to text,concarenating and converting to float
#using iteration
for aux3 in range(3,243,3):
    integer =extracts[aux3-3].text
    decimal =extracts[aux3-2].text
    s_integer =extracts[aux3-1].text
    digit=integer + decimal+s_integer
    digits = float(digit)
    ratings.append(digits)
#importing pandas lib
import pandas as pd
#converting extracted lists to dataframe
movie_parsehub=pd.DataFrame(
    {
     'Movie': movie_titles,
     'Imax screen time':imax_schedule,
     'regular_schedule': regular_schedule,
     'movie_rating': ratings
    })
#splitting regulars_schedule into individual columns
regular_screentimes =movie_parsehub['regular_schedule'].apply(pd.Series)
regular_screentimes= regular_screentimes.rename(columns = lambda x : 'regular_screentime_' + str(x))
#adding new columns to new dataframe
final_parsehub =pd.DataFrame(pd.concat([movie_parsehub[:], regular_screentimes[:]], axis=1))
#dropping regularschedule column
final_parsehub.drop(['regular_schedule'],axis=1,inplace =True)
#creating csv file
final_parsehub.to_csv('my_parse_movie_scrape.csv')